  Obtaining dependency information for enchant from https://files.pythonhosted.org/packages/bb/ea/b7449b0d69361836ed68d0863374c27e395243fd1dbdd5587e62c479535c/enchant-0.0.1-py3-none-any.whl.metadata


In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
import re
import nltk
from nltk.corpus import words

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("NonEnglishWordCount") \
    .getOrCreate()

# Load English words from nltk corpus
english_words = set(words.words())

# Define a function to check if a word is non-English
def is_non_english(word):
    return word.lower() not in english_words

# Define UDF to check if a word is non-English
is_non_english_udf = udf(is_non_english, BooleanType())

# Regular expression pattern to extract words
WORD_RE = re.compile(r"[\w']+")

# Read text file into DataFrame
df = spark.read.text("file2.rtf")

# Split text into words and create DataFrame
words_df = df.selectExpr("explode(split(value, ' ')) as word")

# Filter out non-English words
non_english_words_df = words_df.filter(is_non_english_udf(words_df["word"]))

# Count occurrences of non-English words
non_english_word_counts = non_english_words_df.groupBy("word").count()

# Show non-English word counts
non_english_word_counts.show(1000000000)

# Stop SparkSession
spark.stop()


+--------------------+-----+
|                word|count|
+--------------------+-----+
|            Mrs.\cb1|    2|
|                 ...|    3|
|               ever,|    1|
|               rolls|    1|
|               wand.|    1|
|                now.|    1|
|       \'93Yeah,\'94|    1|
|          vegetables|    1|
|       shrieked.\cb1|    1|
|            \'93\'97|    1|
|           arms.\cb1|    1|
|         wooden.\cb1|    1|
|            sprouted|    1|
|             droopy,|    1|
|         mother\'92s|    1|
|              Lupin,|    1|
|             had\cb1|    1|
|           darkness.|    1|
|        out.\'94\cb1|    1|
|              my\cb1|    1|
|           help!\'94|    1|
|     \'93Sirius?\'94|    1|
|            Hermione|    1|
|              floor.|    1|
|            goblets,|    2|
|              \'93At|    1|
|     habitation,\'94|    1|
|               dived|    1|
|      abandoned.\cb1|    1|
|              jerked|    1|
|           sleepily.|    1|
|             